# **Environment Setting**

## **Install segmentation-models**  
https://github.com/qubvel/segmentation_models

In [1]:
!pip install segmentation-models

## **Link to drive files**

In [2]:
# https://www.jianshu.com/p/000d2a9d36a0

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## **Make sure to use GPU**

In [4]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7959660816372284430
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5064433263838967187
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6803796666904216414
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11066250440838245463
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [5]:
tf.config.list_physical_devices('GPU')
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# **Code section**

In [39]:
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
import tensorflow as tf

import segmentation_models as sm
from segmentation_models import Unet
from keras.models import load_model
from PIL import Image
import skimage.io as io
import skimage.transform as trans

In [8]:
train_path = "/content/drive/Colab/data/DIC-C2DH-HeLa-train/01"
mask_path = "/content/drive/Colab/data/DIC-C2DH-HeLa-train/01_ST/SEG"
test_path = "/content/drive/Colab/data/DIC-C2DH-HeLa/Sequence 1"


# get name lists of training data
train_name_list = []
mask_name_list = []
test_name_list = []

for root, dirs, files in os.walk(train_path, topdown=False):
  for name in files:
    train_name_list.append(name)

for root, dirs, files in os.walk(mask_path, topdown=False):
  for name in files:
    mask_name_list.append(name)

for i in range(10):
  test_name_list.append("t00" + str(i)+ ".tif")

train_name_list = sorted(train_name_list)
mask_name_list = sorted(mask_name_list)
print(len(train_name_list), train_name_list)
print(len(mask_name_list), mask_name_list)
print(len(test_name_list), test_name_list)

84 ['t000.tif', 't001.tif', 't002.tif', 't003.tif', 't004.tif', 't005.tif', 't006.tif', 't007.tif', 't008.tif', 't009.tif', 't010.tif', 't011.tif', 't012.tif', 't013.tif', 't014.tif', 't015.tif', 't016.tif', 't017.tif', 't018.tif', 't019.tif', 't020.tif', 't021.tif', 't022.tif', 't023.tif', 't024.tif', 't025.tif', 't026.tif', 't027.tif', 't028.tif', 't029.tif', 't030.tif', 't031.tif', 't032.tif', 't033.tif', 't034.tif', 't035.tif', 't036.tif', 't037.tif', 't038.tif', 't039.tif', 't040.tif', 't041.tif', 't042.tif', 't043.tif', 't044.tif', 't045.tif', 't046.tif', 't047.tif', 't048.tif', 't049.tif', 't050.tif', 't051.tif', 't052.tif', 't053.tif', 't054.tif', 't055.tif', 't056.tif', 't057.tif', 't058.tif', 't059.tif', 't060.tif', 't061.tif', 't062.tif', 't063.tif', 't064.tif', 't065.tif', 't066.tif', 't067.tif', 't068.tif', 't069.tif', 't070.tif', 't071.tif', 't072.tif', 't073.tif', 't074.tif', 't075.tif', 't076.tif', 't077.tif', 't078.tif', 't079.tif', 't080.tif', 't081.tif', 't082.tif', 

In [44]:
def read_data(image_path,name_list):
  img_list = []
  for i in range(len(name_list)):
    img = cv2.imread(image_path +"/"+ name_list[i],-1)
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    np_img = np.array(img).astype(np.uint8)
    #np_img = np.array(img) 
    np_img = np_img[np.newaxis,:,:]
        
    if i == 0:
      img_list = np_img
    else:
      img_list = np.concatenate((img_list,np_img),axis=0)
    
  return img_list

In [45]:
train_list = read_data(train_path,train_name_list)
train_list = train_list[:,:,:,np.newaxis]
print(train_list.shape)

mask_list = read_data(mask_path,mask_name_list)
mask_list = mask_list[:,:,:,np.newaxis]
print(mask_list.shape)

test_list = read_data(test_path,test_name_list)
test_list = test_list[:,:,:,np.newaxis]
print(test_list.shape)

(84, 512, 512, 1)
(84, 512, 512, 1)
(10, 512, 512, 1)


In [ ]:
model = Unet('resnet34', input_shape = (512, 512, 1), encoder_weights = None)
model.compile('Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(
    x = train_list,
    y = mask_list,
    batch_size = 10,
    epochs = 8,
    validation_split = 0.2,
    shuffle = True
)

model.save("unet.h5")  
print("Saved Unet Model")

In [47]:
model = load_model("unet.h5")
res = model.predict(test_list,5)
print(res.shape)

(10, 512, 512, 1)


In [ ]:
save_path = "/content/drive/Colab/res/"
for i in range(res.shape[0]):
  img = np.squeeze(np.squeeze(res[i,:,:]))
  io.imsave(os.path.join("/content/drive/Colab/res/"+"t00"+str(i)+"_predict_v1.tif"),img)


In [ ]:
img = cv2.imread('/content/drive/Colab/res/t000_predict_v1.tif',-1)
# usable_mask = np.array(mask).astype(np.uint8)
plt.imshow(img)
plt.show()
print(img)
print(img.shape)

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()